#Bibliothèques


- On importe la bibliothèque NLTK qui qui nous aidera concernant le traitement de langage naturel.
- On importe from GENSIM.MODELS les deux models Word2Vec et FastText pour crée notre propre modele.
- On importe plusieurs bibliothéques qui nous aident à réaliser notre model.   

In [ ]:
from sklearn.model_selection import train_test_split
import collections
import numpy as np
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.layers import *
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras import Sequential
from matplotlib import pyplot
from keras.utils.vis_utils import plot_model
import pydot
from keras import optimizers
from keras.layers import Flatten
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec, keyedvectors,FastText
from keras_preprocessing.text import text_to_word_sequence
from bs4 import BeautifulSoup
import re

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Les données

La fonction du nettoyage de texte arabe

In [ ]:
def clean(text):
        # suppression des carac html
        example1 = BeautifulSoup(text, 'lxml')

        # suppression des mentions
        example2 = re.sub(r'@[A-Za-z0-9]+', '', example1.get_text())

        # suppression des liens
        ex3 = re.sub('https?://[A-Za-z0-9./]+', '', example2)

        # suppression des hashtag
        final = re.sub(r'#[A-Za-z0-9]+', "", ex3)
        def remove_emojis(data):
            emoj = re.compile("["
                          u"\U0001f600-\U0001F64F"  # emoticons
                          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                          u"\U0001F680-\U0001F6FF"  # transport & map symbols
                          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          u"\U00002500-\U00002BEF"  # chinese char
                          u"\U00002702-\U000027B0"
                          u"\U00002702-\U000027B0"
                          u"\U000024C2-\U0001F251"
                          u"\U0001f926-\U0001f937"
                          u"\U00010000-\U0010ffff"
                          u"\u2640-\u2642"
                          u"\u2600-\u2B55"
                          u"\u200d"
                          u"\u23cf"
                          u"\u23e9"
                          u"\u231a"
                          u"\ufe0f"  # dingbats
                          u"\u3030"
                          "]+", re.UNICODE)
            return re.sub(emoj, '', data)
        texte = remove_emojis(final)
        return texte

puis on ouvre les fichiers text nécessaires et on lit chaque ligne du fichier ensuite on stocke ces lignes dans une liste, et on fait appel à les fonction de nettoyage

In [ ]:
#Answers
file = open('COVID_Answers_CLASS.txt', 'r', encoding='utf-8')
text = file.read()
Ans = text.split("\n")
A_data = []
for A_da in Ans:
    A_data.append(clean(A_da))
#Question
file = open('COVID_Question.txt', 'r', encoding='utf-8')
text = file.read()
Ques = text.split("\n")
help=Ques[0].replace('\ufeff','')
del Ques[0]
Ques.insert(0,help)
Q_data = []
for Q_da in Ques :
    Q_data.append(clean(Q_da))
file = open('help.txt', 'r', encoding='utf-8')
text = file.read()
Help = text.split("\n")
H_data = []
for H_da in Help :
    H_data.append(clean(H_da))

# Prétraitement

Lors de cette etape ,on divisé les phrases en mots .

In [ ]:
L2 = [text_to_word_sequence(q) for q in Q_data]

Dans cette etape on creée notre modele word2vect, et à l'aide de la fonctions *save_word2vec_format* on stocke notre modèle Word2Vec sous forme de fichier texte

In [ ]:
ft_model2 = Word2Vec(sentences=L2, size=100, window=5, sample=1e-2, negative=10, sg=0, workers=10, min_count=1,iter=20)
ft_model2.wv.save_word2vec_format('COVID.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


**Etape 1: TOKENIZE**


*   Son but c’est de symboliser les données, c'est à dire convertie le texte en valeurs numériques **; transformer chaque chaque mot en nombre ;** 
 

*   Cela permet au réseau neuronal d'effectuer des opérations sur les données d'entrée

In [ ]:
def tokenize(x):
    tokenizer = Tokenizer(char_level=False)
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

*   Exemple :

In [ ]:
text_sentences = ['ما هي أنواع الملابس المتوفرة للصيف','هل يوجد ملابس للصيف.']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'للصيف': 1, 'ما': 2, 'هي': 3, 'أنواع': 4, 'الملابس': 5, 'المتوفرة': 6, 'هل': 7, 'يوجد': 8, 'ملابس': 9}

Sequence 1 in x
  Input:  ما هي أنواع الملابس المتوفرة للصيف
  Output: [2, 3, 4, 5, 6, 1]
Sequence 2 in x
  Input:  هل يوجد ملابس للصيف.
  Output: [7, 8, 9, 1]


**Etape 2: PAD**


*   Lorsque nous introduisons nos séquences d'ID de mot dans le modèle finale, les séquence doient avoir la même longueur.
 

*   Pour ce faire, un remplissage est ajouté à toute séquence plus courte que la longueur maximale (c'est-à-dire plus courte que la phrase la plus longue).

In [ ]:
def pad(x, length=None):
    if length is None:
        length = max([len(sentence) for sentence in x])
        print('Length Max: {}'.format(length))
        print()
    return pad_sequences(x, maxlen=length, padding='post')

*   Exemple (Poursuivre l'exemple précédent)


In [ ]:
test_pad = pad(text_tokenized)

for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Length Max: 6

Sequence 1 in x
  Input:  [2 3 4 5 6 1]
  Output: [2 3 4 5 6 1]
Sequence 2 in x
  Input:  [7 8 9 1]
  Output: [7 8 9 1 0 0]


**Etape 3: Prétraitement**


*   La création d'un dictionnaire des `{mot:id}` pour les Questions et les réponses .

*   Transfomer toutes les Questions et les Réponses à des vecteur.
*   Calcule de la taille du vocabulaire Q et A.

*   la taille maximale des Q et A

In [ ]:
def preprocess(Q):
    preprocess_Q, Q_tk = tokenize(Q)
    preprocess_Q = pad(preprocess_Q)
    return preprocess_Q, Q_tk

Execution

In [ ]:
preproc_Q, Q_tokenizer = preprocess(Q_data)
print(Q_tokenizer.word_index)

max_Q_sequence_length = preproc_Q.shape[1]
Q_vocab_size = len(Q_tokenizer.word_index)+1

print('Data Preprocessed')
print("Max Q sentence length:", max_Q_sequence_length)
print("Q vocabulary size:", Q_vocab_size)

Length Max: 29

{'هل': 1, 'من': 2, 'كورونا': 3, 'كوفيد': 4, '19': 5, 'ما': 6, 'أن': 7, 'جائحة': 8, 'وباء': 9, 'أو': 10, 'هو': 11, 'فيروس': 12, 'في': 13, 'مرض': 14, 'يمكن': 15, 'علاجه': 16, 'هذا': 17, 'كيف': 18, 'مصدر': 19, 'الأمراض': 20, 'هي': 21, 'الفيروس': 22, 'لماذا': 23, 'عن': 24, 'يعتبر': 25, 'ينتقل': 26, 'كغيره': 27, 'كل': 28, 'التهويل': 29, 'والترهيب': 30, 'مثل': 31, 'بما': 32, 'التي': 33, 'إلى': 34, 'أي': 35, 'أدوية': 36, 'تنتقل': 37, 'حيواني': 38, 'الجديد': 39, 'فيروسي': 40, 'طريق': 41, 'البشر': 42, 'المضادات': 43, 'الحيوية': 44, 'فعّالة': 45, 'الوقاية': 46, 'توجد': 47, 'علاجات': 48, 'يمكنها': 49, 'الحماية': 50, 'الفيروسية': 51, 'الأخرى': 52, 'بمرض': 53, 'أصاب': 54, 'الطعام': 55, 'الإصابة': 56, 'على': 57, 'الإيدز': 58, 'مصاب': 59, 'عبر': 60, 'قبل': 61, 'أعراض': 62, 'بالمرض': 63, 'بفيروس': 64, 'هناك': 65, 'شخص': 66, 'علامات': 67, 'المرض': 68, 'كان': 69, 'الجسم': 70, 'لو': 71, 'الحساسية': 72, 'الأرجية': 73, 'تعد': 74, 'المزمنة': 75, 'يهدد': 76, 'صاحبها': 77, 'ينتشر': 78, 'السن':





*   Création de vecteue one-hot des réponses





In [ ]:
Y=[]
for q in A_data:
    a=[0]*81
    a[int(q)]=1
    Y.append(a)

**Etape 4: Embbedings**


*   l’utilisation d'une matrice qui contient les représentations des mots en vecteurs réalisés avec plusieurs méthodes **(W2V )** ,donc ça sera nécessaire de remplacer chaque mot par un vecteur comme input du model.


*   Et Dans notre cas la taille de chaque vecteur est 100

In [ ]:
embeddings_index = {}
f = open('COVID.txt',encoding='utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()


def embedding_matrix_creater(embedding_dimention):
  embedding_matrix = np.zeros((Q_vocab_size, embedding_dimention))
  for word, i in Q_tokenizer.word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector
  return embedding_matrix

mat=embedding_matrix_creater(100)

# Modèle

L'utilisation d'un modèle de classification, LSTM  et des couches dense avec une sortie  de 81 classe  qui ont des probabilités differentes on utlisant la fonction d'activation softmax.   

In [ ]:
input_length = max_Q_sequence_length
embedding_dim = 100

Q_train,Q_test,A_train,A_test=train_test_split(preproc_Q,Y,test_size=0.05,random_state=10)

model = Sequential()
model.add(Embedding(Q_vocab_size, embedding_dim,
            input_length = input_length,weights=[mat] , trainable = True))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(64, activation = "relu"))
model.add(Dense(81, activation = "relu"))
model.add(BatchNormalization())
model.add(Dense(np.array(Y).shape[1], activation = "softmax"))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 29, 100)           41800     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               234496    
_________________________________________________________________
dense_10 (Dense)             (None, 64)                16448     
_________________________________________________________________
dense_11 (Dense)             (None, 81)                5265      
_________________________________________________________________
batch_normalization_4 (Batch (None, 81)                324       
_________________________________________________________________
dense_12 (Dense)             (None, 81)                6642      
Total params: 304,975
Trainable params: 304,813
Non-trainable params: 162
______________________________________________

L'éxecution de l'entrainement necessite deux fonction. model.compile qui aide le modèle à corriger durant l'entrainement plusieurs paramétres, et model.fit qui lance l'entrainement avec les données d'entrées et d'autres paramétres .

In [ ]:
rms = optimizers.RMSprop(lr=0.001)
model.compile(optimizer=rms, loss='categorical_crossentropy',metrics=['accuracy'])

history = model.fit(np.array(preproc_Q),np.array(Y), 
          epochs=100,batch_size=32,
          )

Evaluation du model 

In [ ]:
[a,b]=model.evaluate(np.array(Q_test),np.array(A_test))
print('loss : {} , accuracy: {}'.format(a,b))